In [10]:
import numpy as np
import cupy as cp
import numba as nb
from numba import cuda
from sympy import *
import inspect

In [33]:
# define symbols
x = Symbol('x')
y = cos(x) * exp(-x**2 / 25)
f = lambdify(x, y, 'numpy')
dy_x = diff(y, x)
simplify(dy_x)
df = lambdify(x, dy_x, 'numpy')

In [28]:
inspect.getsource(lambdify(x, dy_x, 'numpy'))

'def _lambdifygenerated(x):\n    return -2/25*x*exp(-1/25*x**2)*cos(x) - exp(-1/25*x**2)*sin(x)\n'

In [32]:
inspect.getsource(lambdify(x, y, 'numpy')).split("return ")[1].split("\n")[0]

'exp(-1/25*x**2)*cos(x)'

In [36]:
print(f'{dy_x}')

-2*x*exp(-x**2/25)*cos(x)/25 - exp(-x**2/25)*sin(x)


In [22]:
n = 1024
L = 30
dx = L/n
X = np.arange(-L/2, L/2, dx, dtype=np.complex128)
k = np.fft.fftshift((2*np.pi/L) * np.arange(-n/2, n/2))

u = np.cos(X) * np.exp(-X**2/25)
df = -np.sin(X) * np.exp(-X**2/25) -2/25*X*f


In [41]:
u = f(X)
du = df(X)

In [29]:
def fft_derivative(u):
    uhat = np.fft.fft(u)
    duhat = k * uhat * (1.j)
    duFFT = np.fft.ifft(duhat).real
    return duFFT

In [38]:
duFFT = fft_derivative(u)

In [40]:
# plot A with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=u.real, mode='lines', name='real'))
# fig.add_trace(go.Scatter(x=X, y=A.imag, mode='lines', name='imag'))

# title
fig.update_layout(title=f'f(x) = {y}', xaxis_title='X', yaxis_title='A')
fig.show()

In [48]:
# plot A with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=du.real, mode='lines', name='analytic'))
fig.add_trace(go.Scatter(x=X.real, y=duFFT.real, mode='lines', name='FFT'))
# fig.add_trace(go.Scatter(x=X, y=A.imag, mode='lines', name='imag'))

# title
fig.update_layout(title=f'f\'(x) = {dy_x}', xaxis_title='X', yaxis_title='A')
fig.show()

In [53]:
# plot the difference between du and duFFT
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=du.real - duFFT.real, mode='lines', name='analytic - FFT'))

# title
fig.update_layout(title='f\'(x) - F^{-1}[i*k*F[f(x)]]', xaxis_title='X', yaxis_title='A')
fig.show()

In [54]:
# time the fft derivative
%timeit fft_derivative(u)

30.6 μs ± 2.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [80]:
import numba as nb
import numpy as np

@cuda.jit
def jit_fft(x, xk):
    xk[:] = np.fft.fft(x)

a = np.array([1, 6, 1, 8, 0, 3, 3, 9])

In [82]:
jit_fft[1,1](a, a)

d:\GitHubWSU\pfc-blocks\.venv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning:

Grid size 1 will likely result in GPU under-utilization due to low occupancy.



TypeError: Signature mismatch: 1 argument types given, but function takes 6 arguments

In [77]:
cp.fft.fft(cp.asarray(u))

array([  0.5893046 +0.00000000e+00j,  -3.05365217+2.25185278e-15j,
        18.33124441+2.34206008e-15j, ..., -63.77956048+4.85351561e-15j,
        18.33124441-4.12088220e-15j,  -3.05365217-2.04442930e-15j])

In [72]:
import numpy as np
import cupy as cp
import numba as nb
from numba import cuda

@cuda.jit
def custom_fft(A, Ak):
    # Perform operations on Ak (modify Ak as needed)
    # Example: Ak *= 2

    # Compute FFT of Ak using CuPy
    Ak_fft = cp.fft.fft(Ak)

    # Perform additional operations on Ak_fft
    # Example: Ak_fft += 1
    Ak_fft += 1

    # Compute inverse FFT to get A
    A[:] = cp.fft.ifft(Ak_fft)

In [74]:
A = np.arange(8)
Ak = np.zeros_like(A)

# Transfer arrays to GPU
A_gpu = cp.asarray(A)
Ak_gpu = cp.asarray(Ak)

# Call the custom kernel
#custom_fft, 1
custom_fft[1, 1](A_gpu, Ak_gpu)

# Transfer result back to CPU
A_result = cp.asnumpy(A_gpu)

print("Modified A:")
print(A_result)

d:\GitHubWSU\pfc-blocks\.venv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning:

Grid size 1 will likely result in GPU under-utilization due to low occupancy.



TypingError: Failed in cuda mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'fft' of type Module(<module 'cupy.fft' from 'd:\\GitHubWSU\\pfc-blocks\\.venv\\Lib\\site-packages\\cupy\\fft\\__init__.py'>)
[1m
File "C:\Users\estim\AppData\Local\Temp\ipykernel_57856\3398067964.py", line 12:[0m
[1mdef custom_fft(A, Ak):
    <source elided>
    # Compute FFT of Ak using CuPy
[1m    Ak_fft = cp.fft.fft(Ak)
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of get attribute at C:\Users\estim\AppData\Local\Temp\ipykernel_57856\3398067964.py (12)[0m
[1m
File "C:\Users\estim\AppData\Local\Temp\ipykernel_57856\3398067964.py", line 12:[0m
[1mdef custom_fft(A, Ak):
    <source elided>
    # Compute FFT of Ak using CuPy
[1m    Ak_fft = cp.fft.fft(Ak)
[0m    [1m^[0m[0m


In [62]:
@cuda.jit
def cuda_fft(A, Ak):
    # Perform operations on Ak (modify Ak as needed)
    # Example: Ak *= 2

    # Compute FFT of Ak using CuPy
    Ak[:] = cp.fft.fft(A)


In [ ]:
_u = cp.asarray(u)
_uk = cp.zeros_like(_u)
cuda_fft[1, 1](_u, _uk)
_uk

In [ ]:
@cuda.jit
def cuda_ifft(A, Ak):
    # Perform operations on Ak (modify Ak as needed)
    # Example: Ak *= 2

    # Compute FFT of Ak using CuPy
    Ak_fft = cp.fft.fft2(Ak)

    # Perform additional operations on Ak_fft
    # Example: Ak_fft += 1

    # Compute inverse FFT to get A
    A[:] = cp.fft.ifft2(Ak_fft)

In [59]:
def cuda_fft_derivative(u):
    uhat = np.fft.fft(u)
    duhat = k * uhat * (1.j)
    duFFT = np.fft.ifft(duhat).real
    return duFFT

In [83]:
import cupy as cp
import cupyx.scipy.fft as cufft
import scipy.fft

a = cp.random.random(100).astype(cp.complex64)
with scipy.fft.set_backend(cufft):
    b = scipy.fft.fft(a)  # equivalent to cufft.fft(a)

ModuleNotFoundError: No module named 'scipy'